<a href="https://colab.research.google.com/github/01star01ek/01star01ek/blob/main/Real-ESRGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf /content/results

In [1]:
# ▶️ Git 클론 및 환경 설치
%cd /content
!git clone https://github.com/woctezuma/SimSwap.git
%cd /content/SimSwap
!git checkout upgrade-insightface

!pip install insightface==0.7.3 onnxruntime moviepy > /dev/null
!pip install imageio==2.34.0 > /dev/null

# ▶️ 모델 및 리소스 다운로드
!wget https://github.com/woctezuma/SimSwap-colab/releases/download/antelope/antelope.zip
!wget -P ./arcface_model https://github.com/woctezuma/SimSwap-colab/releases/download/1.0/arcface_checkpoint.tar
!wget https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip
!wget -P ./parsing_model/checkpoint https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth
!wget https://github.com/neuralchen/SimSwap/releases/download/512_beta/512.zip

# ▶️ 압축 해제
!unzip -oq checkpoints.zip -d ./checkpoints
!unzip -oq 512.zip -d ./checkpoints
!unzip -oq antelope.zip -d ./insightface_func/models/


/content
Cloning into 'SimSwap'...
remote: Enumerating objects: 1143, done.
remote: Counting objects: 100% (505/505), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 1143 (delta 417), reused 386 (delta 386), pack-reused 638 (from 1)
Receiving objects: 100% (1143/1143), 211.46 MiB | 13.91 MiB/s, done.
Resolving deltas: 100% (601/601), done.
/content/SimSwap
Already on 'upgrade-insightface'
Your branch is up to date with 'origin/upgrade-insightface'.
--2025-06-17 00:36:55--  https://github.com/woctezuma/SimSwap-colab/releases/download/antelope/antelope.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/378697706/282dd755-3c47-466c-b3e5-be6e4d9a491f?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250617%2Fus-east-1%2Fs3%2Faws4_requ

In [2]:
# ▶️ SimSwap 경로 설정
%cd /content/SimSwap

import sys
sys.path.append('/content/SimSwap')

import os
import cv2
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from models.models import create_model
from options.test_options import TestOptions
from insightface_func.face_detect_crop_multi import Face_detect_crop
from util.videoswap import video_swap

# ▶️ PyTorch 2.6+ 보안 대응 - 모델 클래스 허용
from torch.serialization import add_safe_globals
from models.arcface_models import ResNet, IRBlock, SEBlock
from torch.nn import (
    Conv2d, BatchNorm2d, BatchNorm1d, PReLU, MaxPool2d, Sequential,
    ReLU, Linear, AdaptiveAvgPool2d, Dropout, Module, Sigmoid
)

add_safe_globals([
    Conv2d, BatchNorm2d, BatchNorm1d, PReLU, MaxPool2d, Sequential,
    ReLU, Linear, AdaptiveAvgPool2d, Dropout, Module, Sigmoid,
    ResNet, IRBlock, SEBlock
])


/content/SimSwap


  if event.key is 'enter':



In [3]:
# ▶️ 기본 설정
crop_size = 512  # 정확도 유지용
det_thresh = 0.5
det_size = (320, 320)

# ▶️ 얼굴 임베딩용 변환기
transformer_Arcface = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [4]:
# ▶️ SimSwap 옵션 설정
opt = TestOptions()
opt.initialize()
opt.parser.add_argument('-f')  # for Colab compatibility
opt = opt.parse()
opt.isTrain = False
opt.no_simswaplogo = True
opt.use_mask = True
opt.crop_size = crop_size
opt.Arc_path = './arcface_model/arcface_checkpoint.tar'
opt.name = '512' if crop_size == 512 else 'people'
opt.which_epoch = 550000 if crop_size == 512 else 'latest'
mode = 'ffhq' if crop_size == 512 else 'None'

# ▶️ 모델 생성 및 얼굴 인식기 준비
torch.nn.Module.dump_patches = True
model = create_model(opt)
model.eval()

app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id=0, det_thresh=det_thresh, det_size=det_size, mode=mode)


------------ Options -------------
Arc_path: arcface_model/arcface_checkpoint.tar
aspect_ratio: 1.0
batchSize: 8
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
crop_size: 512
data_type: 32
dataroot: ./datasets/cityscapes/
display_winsize: 512
engine: None
export_onnx: None
f: /root/.local/share/jupyter/runtime/kernel-df8b0dac-1351-4d14-b035-df10f5d7d866.json
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 50
id_thres: 0.03
image_size: 224
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
latent_size: 512
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
multisepcific_dir: ./demo_file/multispecific
nThreads: 2
n_blocks_global: 6
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 3
n_local_enhancers: 1
name: people
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: False
no_simswaplogo: False
norm: batch
norm_G: spectralspadesyncbatch3x3
ntest: inf
onnx: 

  warnings.warn(



Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (320, 320)


In [5]:
import os
import cv2
from PIL import Image
import torch.nn.functional as F

# ▶️ 입력/출력 경로
video_dir = '/content/videos'
photo_dir = '/content/photos'
output_root = '/content/results'

# ▶️ 파일 목록 (정렬된 순서로 매칭)
video_files = sorted([f for f in os.listdir(video_dir) if f.endswith('.mp4')])
photo_files = sorted([f for f in os.listdir(photo_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

# ▶️ 순서 기준 매칭
num_files = min(len(video_files), len(photo_files))

for i in range(num_files):
    video_file = video_files[i]
    photo_file = photo_files[i]

    video_id = str(i + 1).zfill(4)  # 0001, 0002, ...
    temp_path = os.path.join(output_root, f"tmp_{video_id}")
    output_path = os.path.join(temp_path, f"result_{video_id}.mp4")  # ✅ tmp 폴더 안에 저장

    # ✅ temp 폴더 생성
    os.makedirs(temp_path, exist_ok=True)

    # ✅ 실제 경로 지정
    input_video = os.path.join(video_dir, video_file)
    photo_path = os.path.join(photo_dir, photo_file)

    opt.video_path = input_video
    opt.pic_a_path = photo_path
    opt.output_path = output_path
    opt.temp_path = temp_path

    # ▶ 얼굴 임베딩 생성
    img_a_whole = cv2.imread(photo_path)
    result = app.get(img_a_whole, opt.crop_size)

    if result is None:
        h, w, _ = img_a_whole.shape
        center = (w // 2, h // 2)
        half = opt.crop_size // 2
        cropped = img_a_whole[max(0, center[1]-half):center[1]+half,
                              max(0, center[0]-half):center[0]+half]
        img_pil = Image.fromarray(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
    else:
        img_align, _ = result
        img_pil = Image.fromarray(cv2.cvtColor(img_align[0], cv2.COLOR_BGR2RGB))

    img_a = transformer_Arcface(img_pil).unsqueeze(0).cuda()
    img_id_downsample = F.interpolate(img_a, size=(112, 112))
    latend_id = model.netArc(img_id_downsample)
    latend_id = F.normalize(latend_id, p=2, dim=1)

    # ▶ 얼굴 교체 실행
    video_swap(input_video, latend_id, model, app, output_path,
               temp_results_dir=temp_path,
               no_simswaplogo=opt.no_simswaplogo,
               use_mask=opt.use_mask,
               crop_size=opt.crop_size)

    print(f"[✅] 완료: {video_id} → {output_path}")


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


(142, 366, 4)


100%|██████████| 44.7M/44.7M [00:00<00:00, 332MB/s]
  0%|          | 0/469 [00:00<?, ?it/s]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

100%|██████████| 469/469 [02:30<00:00,  3.11it/s]


Moviepy - Building video /content/results/tmp_0001/result_0001.mp4.
Moviepy - Writing video /content/results/tmp_0001/result_0001.mp4



Moviepy - Done !
Moviepy - video ready /content/results/tmp_0001/result_0001.mp4
[✅] 완료: 0001 → /content/results/tmp_0001/result_0001.mp4
(142, 366, 4)


...

Moviepy - Building video /content/results/tmp_0003/result_0003.mp4.
Moviepy - Writing video /content/results/tmp_0003/result_0003.mp4



Moviepy - Done !
Moviepy - video ready /content/results/tmp_0003/result_0003.mp4
[✅] 완료: 0003 → /content/results/tmp_0003/result_0003.mp4
(142, 366, 4)


100%|██████████| 529/529 [02:40<00:00,  3.30it/s]


Moviepy - Building video /content/results/tmp_0004/result_0004.mp4.
Moviepy - Writing video /content/results/tmp_0004/result_0004.mp4



Moviepy - Done !
Moviepy - video ready /content/results/tmp_0004/result_0004.mp4
[✅] 완료: 0004 → /content/results/tmp_0004/result_0004.mp4


In [6]:
import os
from glob import glob

# 🔧 경로 설정
output_root = "/content/results"
tmp_dirs = sorted(glob(os.path.join(output_root, "tmp_*")))

print(f"총 {len(tmp_dirs)}개 영상 후처리 시작")

for i, tmp_dir in enumerate(tmp_dirs):
    base_id = tmp_dir.split("_")[-1]  # 'tmp_0001' → '0001'

    result_path  = os.path.join(tmp_dir, f"result_{base_id}.mp4")
    denoise_path = os.path.join(tmp_dir, f"denoise_{base_id}.mp4")
    sharpen_path = os.path.join(tmp_dir, f"sharpen_{base_id}.mp4")
    final_path   = os.path.join(tmp_dir, f"final_{base_id}.mp4")

    print(f"\n▶️ [{i+1}/{len(tmp_dirs)}] result_{base_id}.mp4 후처리 중...")

    if not os.path.exists(result_path):
        print(f"⚠️ result_{base_id}.mp4 없음 → 건너뜀")
        continue

    # 노이즈 제거
    print(" - 노이즈 제거 중...")
    os.system(f'ffmpeg -y -i "{result_path}" -vf "hqdn3d=4.0:3.0:6.0:4.5" -c:a copy "{denoise_path}"')

    # 선명화
    print(" - 선명화 중...")
    os.system(f'ffmpeg -y -i "{denoise_path}" -vf "unsharp=5:5:1.0:5:5:0.0" -c:a copy "{sharpen_path}"')

    # 최종 출력 (30fps + 고화질 압축)
    print(" - 최종 인코딩 중...")
    os.system(f'ffmpeg -y -i "{sharpen_path}" -filter:v "fps=30" -c:v libx264 -preset slow -crf 18 -pix_fmt yuv420p "{final_path}"')

    print(f"✅ 저장 완료 → {final_path}")


총 4개 영상 후처리 시작

▶️ [1/4] result_0001.mp4 후처리 중...
 - 노이즈 제거 중...
 - 선명화 중...
 - 최종 인코딩 중...
✅ 저장 완료 → /content/results/tmp_0001/final_0001.mp4

▶️ [2/4] result_0002.mp4 후처리 중...
 - 노이즈 제거 중...
 - 선명화 중...
 - 최종 인코딩 중...
✅ 저장 완료 → /content/results/tmp_0002/final_0002.mp4

▶️ [3/4] result_0003.mp4 후처리 중...
 - 노이즈 제거 중...
 - 선명화 중...
 - 최종 인코딩 중...
✅ 저장 완료 → /content/results/tmp_0003/final_0003.mp4

▶️ [4/4] result_0004.mp4 후처리 중...
 - 노이즈 제거 중...
 - 선명화 중...
 - 최종 인코딩 중...
✅ 저장 완료 → /content/results/tmp_0004/final_0004.mp4


In [5]:
# NumPy 버전 다운 (중요!)
!pip install numpy==1.24.4

# Real-ESRGAN 초기화
%cd /content
!rm -rf Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
!pip install basicsr facexlib gfpgan
!pip install -r requirements.txt
!python setup.py develop

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.
/content
Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 759 (delta 106), reused 99 (delta 99), pack-reused 638 (from 1)
Receiving objects: 100% (759/759), 5.38 MiB | 10.93 MiB/s, done.
Resolving deltas: 100% (415/415), done.
/content/Real-ESRGAN
/usr/local/lib/python3.11/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are us

In [6]:
import os

os.makedirs("video_upload", exist_ok=True)
os.makedirs("frames_raw", exist_ok=True)
os.makedirs("frames_sr", exist_ok=True)

In [7]:
from google.colab import files
uploaded = files.upload()

# 예: 비디오 저장
import shutil
for name in uploaded:
    shutil.move(name, f"video_upload/{name}")


Saving final_0001 (2).mp4 to final_0001 (2).mp4


In [10]:

import os
os.rename("video_upload/final_0001 (2).mp4", "video_upload/final_0001.mp4")


In [11]:
!ffmpeg -i video_upload/final_0001.mp4 frames_raw/frame_%05d.png

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [13]:
!pip install numpy==1.24.4 --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 106.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.4 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
pymc 5.23.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is

In [3]:
%cd /content/Real-ESRGAN
!python inference_realesrgan.py -n RealESRGAN_x4plus -i frames_raw --outscale 3.5 --face_enhance

/content/Real-ESRGAN
/usr/local/lib/python3.11/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
Downloading: "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth" to /content/Real-ESRGAN/weights/RealESRGAN_x4plus.pth

100% 63.9M/63.9M [00:00<00:00, 289MB/s]
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the 

In [5]:
!ffmpeg -framerate 30 -i /content/frames_raw_001/frame_%05d.png -c:v libx264 -pix_fmt yuv420p result_upscaled.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [6]:
from google.colab import files
files.download('result_upscaled.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>